In [1]:
import utrain

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


How much epochs during which we choose to train our model. We choose 100 to compare the difference between our parameters. This allows for enough training time to compare the differences of the choice of those parameters, assuming improvements on 100 epochs hold for more epochs.

In [2]:
N_EPOCHS = 1
N_STEPS_PER_EPOCH = 1

### Baseline model

This is the choice of parameters for the baseline model. We minimize the number of weights using a smaller image size (from 400\*400 resampled to 256\*256), and using 1 grayscale channel to aggregate the 3 color channels.

In [ ]:
utrain.main(
    img_height=256,
    batch_size=2,
    epochs=N_EPOCHS,
    steps_per_epoch=N_STEPS_PER_EPOCH,
    aug=False,
    chosen_validation=False,
    rgb=False,
    pretrained_weights=None,
    monitor="val_acc",
    root_folder="."
)

### Original image size

We try to use the original image size instead. This eliminates the resampling process, but results in a larger number of weights to train thus longer training times. Note that resampling to a smaller size condenses more "context" information for each pixel : the convolution kernel size is fixed, therefore a 256\*256 resampling provides to the first convolution layer 400/256 more pixel information to the kernel (if we ignore the loss of information during resampling). Nevertheless, we evaluate our model performance on the original image size.


In [ ]:
utrain.main(
    img_height=400,
    batch_size=2,
    epochs=N_EPOCHS,
    steps_per_epoch=N_STEPS_PER_EPOCH,
    aug=True,
    chosen_validation=False,
    rgb=False,
    pretrained_weights=None,
    monitor="val_acc",
    root_folder="."
)

### Using dataset augmentation

Now, we apply dataset augmentation using Keras' preprocessing ImageDataGenerator class. We use it, by default, to transform the original dataset images using rotations and flips. This could proove useful because most of the training dataset roads are axis-aligned, and tilted roads could be harder for our model to segment.

In [ ]:
utrain.main(
    img_height=256,
    batch_size=2,
    epochs=N_EPOCHS,
    steps_per_epoch=N_STEPS_PER_EPOCH,
    aug=True,
    chosen_validation=False,
    rgb=False,
    pretrained_weights=None,
    monitor="val_acc",
    root_folder="."
)

### Using red, green and blue channels

Instead of using the grayscale of the training images, we train using directly the separate channels. This results in more weights to train thus longer training times, but also preserves more original informations about the training images.

In [ ]:
utrain.main(
    img_height=256,
    batch_size=2,
    epochs=N_EPOCHS,
    steps_per_epoch=N_STEPS_PER_EPOCH,
    aug=True,
    chosen_validation=False,
    rgb=True,
    pretrained_weights=None,
    monitor="val_acc",
    root_folder="."
)

### Using the chosen validation dataset

Instead of letting the choice of the separation of the training and validation datasets to randomness, we use a custom predetermined split. This can be useful because some training images are more exotic than others, providing more useful information to our neural network. Would it happen that most/all of those be chosen for validation, this useful information would not be used to train our network. Instead, we set a training dataset with some conventional images and also some exotic images, and a validation dataset also with some conventional and exotic images. The details of the chosen indicies can be found in the common.py file.

In [ ]:
utrain.main(
    img_height=256,
    batch_size=2,
    epochs=N_EPOCHS,
    steps_per_epoch=N_STEPS_PER_EPOCH,
    aug=True,
    chosen_validation=True,
    rgb=False,
    pretrained_weights=None,
    monitor="val_acc",
    root_folder="."
)

# Using ImageDataGenerator parameters

We try changing the parameters of ImageDataGenerator, changing rotation, adding zoom and shift. The missing pixels in each case, if any, are filled using the reflected method.

### Rotation

We try adding more rotation to ImageDataGenerator, from 90 to 360.

In [ ]:
dgargs = dict(    
    rotation_range=360, # instead of 90
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
)
utrain.main(
    img_height=256,
    batch_size=2,
    epochs=N_EPOCHS,
    steps_per_epoch=N_STEPS_PER_EPOCH,
    aug=True,
    chosen_validation=True,
    rgb=False,
    pretrained_weights=None,
    monitor="val_acc",
    root_folder=".",
    data_gen_args=dgargs
)

### Shift

We try using shifting in the x and y axis, with a maximum shift of 20% the width/height of the image.

In [4]:
dgargs = dict(    
    rotation_range=90,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
)
utrain.main(
    img_height=256,
    batch_size=2,
    epochs=5,
    steps_per_epoch=10,
    aug=True,
    chosen_validation=True,
    rgb=False,
    pretrained_weights=None,
    monitor="val_acc",
    root_folder=".",
    data_gen_args=dgargs
)

[ENV] No missing folder in the environement.
[MODEL] Using kernel initializer he_uniform with seed 2018


/home/louis/Documents/EPFL/ML/group/ML_course/projects/project2/project_road_segmentation/unet/model.py:87: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=inputs, output=conv10)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 256, 256, 64) 640         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_25[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_26[0][0]                  
__________________________________________________________________________________________________
conv2d_27 

Found 80 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
hdf5 name: unet_bw_256_2018-12-19_15_15_14.055643.hdf5
Epoch 1/5
10/10 [==============================] - 181s 18s/step - loss: 0.5368 - acc: 0.8148 - val_loss: 0.5134 - val_acc: 0.7947

Epoch 00001: val_acc improved from -inf to 0.79467, saving model to results/unet_bw_256_2018-12-19_15_15_14.055643.hdf5
Epoch 2/5
10/10 [==============================] - 202s 20s/step - loss: 0.5168 - acc: 0.7897 - val_loss: 0.5100 - val_acc: 0.7947

Epoch 00002: val_acc did not improve from 0.79467
Epoch 3/5
10/10 [==============================] - 204s 20s/step - loss: 0.4844 - acc: 0.8131 - val_loss: 0.5098 - val_acc: 0.7947

Epoch 00003: val_acc did not improve from 0.79467
Epoch 4/5
10/10 [==============================] - 209s 21s/step - loss: 0.5523 - acc: 0.7704 - val_loss: 0.5047 - val_acc: 0.7947

Epoch 00004: val_acc did not improve from 0.79467
Epoch 5/5
10/10 [============

'results/unet_bw_256_2018-12-19_15_15_14.055643.hdf5'

### Zoom

We try using zoom augmentation, of 20% the image current scale.

In [ ]:
dgargs = dict(    
    rotation_range=90,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
)
utrain.main(
    img_height=256,
    batch_size=2,
    epochs=N_EPOCHS,
    steps_per_epoch=N_STEPS_PER_EPOCH,
    aug=True,
    chosen_validation=True,
    rgb=False,
    pretrained_weights=None,
    monitor="val_acc",
    root_folder=".",
    data_gen_args=dgargs
)